In [1]:
import warnings
warnings.filterwarnings("ignore")

In [16]:
import scanpy as sc
import numpy as np
import torch
from torch.utils.data import DataLoader
from arch1 import Arch1
from dataset import MyDataset
from utils import train


In [17]:
file = ''
gdata = sc.read_visium(file, count_file=r'C:\Users\KARAN\Desktop\MultiOmics-Research\STAGATE\Landau\SPOTS Landau paper dataset\protein\GSE198353_mmtv_pymt_GEX_filtered_feature_bc_matrix.h5',load_images=True)
gdata.var_names_make_unique()

In [18]:
pdata = sc.read_csv(r'C:\Users\KARAN\Desktop\MultiOmics-Research\STAGATE\Landau\SPOTS Landau paper dataset\protein\GSE198353_mmtv_pymt_ADT_t.csv')
pdata.var_names_make_unique()

In [19]:
gdata

AnnData object with n_obs × n_vars = 1978 × 32286
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [20]:
pdata

AnnData object with n_obs × n_vars = 1978 × 32

In [21]:
import numpy as np

def custom_collate_fn(batch):
    genes_batch, proteins_batch = zip(*batch)
    return list(genes_batch), list(proteins_batch)


In [22]:
batch_size = 32

dataset = MyDataset(gdata, pdata)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)


In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [24]:
latent_dim = 32
learning_rate = 0.001
num_epochs = 100


In [25]:
from utils import compute_spatial_graph

In [26]:
radius = 10.0
spatial_graph = compute_spatial_graph(gdata.obsm['spatial'], radius)
model = Arch1(gdata.shape[1], pdata.shape[1], latent_dim, spatial_graph).to(device)


In [27]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train_loss = train(model, optimizer, dataloader,num_epochs)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")


TypeError: Could not generate a generic TraceType for <32x32286 sparse matrix of type '<class 'numpy.float32'>'
	with 126678 stored elements in Compressed Sparse Row format>.Please verify that it is immutable/hashable. Otheriwse, consider implementing the Tracing Protocol for it.

In [ ]:
for epoch in range(num_epochs):
    train_loss = train(model, optimizer, dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}")


NameError: name 'optimizer' is not defined

In [ ]:
with torch.no_grad():
    evaluate(model, dataloader)


In [ ]:
import numpy as np
from sklearn.cluster import KMeans

# Load the learned latent variables
latent_variables = model.encode(genes_data, proteins_data)

# Perform clustering using K-means
kmeans = KMeans(n_clusters=5)  # Specify the number of clusters
cluster_labels = kmeans.fit_predict(latent_variables)

# Print cluster labels
print(cluster_labels)
